In [2]:
from sage.algebras.flag_algebras import *

In [5]:
def blowup_construction(self, target_size, construction_size, **kwargs):
    from tqdm import tqdm
    res = -1
    for verts in tqdm(itertools.product(range(construction_size), repeat=int(target_size))):
        #print(verts)
        blocks = {}
        for rel in kwargs:
            if rel not in self.signature():
                continue
            reledges = kwargs[rel]
            bladd = []
            for edge in reledges:
                clusters = [[ii for ii in range(target_size) if verts[ii]==ee] for ee in edge]
                bladd += [xx for xx in itertools.product(*clusters) if len(set(xx))==len(edge) and xx[0]==min(xx) and xx[-1]==max(xx)]
            blocks[rel] = bladd
        #print(self(target_size, **blocks))
        res += self(target_size, **blocks)
    return (res+1)/(construction_size**target_size)

def sym_asym_bases(self, n, ftype=None):
    r"""
    Generate the change of base matrices for the symmetric
    and the asymmetric subspaces
    """
    
    flags = self.generate_flags(n, ftype)
    uniques = []
    sym_base = []
    asym_base = []
    for xx in flags:
        xxid = self.identify(n, [xx.ftype()], **xx.blocks())
        if xxid not in uniques:
            uniques.append(xxid)
            sym_base.append(xx.afae())
        else:
            sym_ind = uniques.index(xxid)
            asym_base.append(sym_base[sym_ind] - xx.afae())
            sym_base[sym_ind] += xx
    m_sym = matrix(len(sym_base), len(flags), [xx.values() for xx in sym_base], sparse=True)
    m_asym = matrix(len(asym_base), len(flags), [xx.values() for xx in asym_base], sparse=True)
    return m_sym, m_asym

#for quick sparse matrix printing
def print_sparse(ls, eps=1e-6):
    nzs = [(ii, ls[ii]) for ii in range(len(ls)) if abs(ls[ii])>eps]
    if isinstance(ls[0], Rational):
        st = "\n".join(["{}: {}".format(nn[0], nn[1]) for nn in nzs])
    else:
        st = "\n".join(["{}: {:.4f}".format(nn[0], float(nn[1])) for nn in nzs])
    return st

#continued fraction rounding
def cfr(value, quotient_bound=7, denom_bound=9):
    cf = continued_fraction(value)
    for ii, xx in enumerate(cf.quotients()):
        if xx>=2**quotient_bound or cf.denominator(ii)>2**(denom_bound):
            if ii>0:
                return cf.convergent(ii-1)
            return 0
    return cf.value()

#rounding for lists
def cfr_list(ls, force_pos=False):
    if force_pos:
        return [max(cfr(xx), 0) for xx in ls]
    else:
        return [cfr(xx) for xx in ls]

#rounding for matrices
def cfr_matrix(mat):
    return matrix(QQ, [cfr_list(xx, False) for xx in mat])

#rounding matrix based on LDL factoring
def cfr_ldl(mat):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, cfr_matrix(mat_ldl[1]))
    D = diagonal_matrix(QQ, cfr_list(mat_ldl[2].diagonal(), True))
    pl = P*L
    return pl*D*pl.T

#rounding based on simple denominator scaling
def sdem(value, den=1024):
    return floor(value*den)/den

#rounding for lists
def sdem_list(ls, force_pos=False):
    if force_pos:
        return [max(sdem(xx), 0) for xx in ls]
    else:
        return [sdem(xx) for xx in ls]

#rounding for matrices
def sdem_matrix(mat):
    return matrix(QQ, [sdem_list(xx, False) for xx in mat])

#rounding matrix based on LDL factoring
def sdem_ldl(mat):
    mat_ldl = matrix(mat).block_ldlt()
    P = matrix(QQ, mat_ldl[0])
    L = matrix(QQ, sdem_matrix(mat_ldl[1]))
    D = diagonal_matrix(QQ, sdem_list(mat_ldl[2].diagonal(), True))
    pl = P*L
    return pl*D*pl.T

#adaptive continued fraction rounding, seems to work well
def cfr_adaptive(ls, onevec):
    best_vec = None
    best_error = 1000
    best_lcm = 1000000000
    
    orig = vector(ls)
    for resol1 in range(5, 20):
        resol2 = round(resol1*1.5)
        rls = vector([cfr(xx, resol1, resol2) for xx in ls])
        ip = rls*onevec
        if ip != 0 and abs(ip - 1)<best_error:
            if ip.as_integer_ratio()[1] > best_lcm**1.5 and ip != 1:
                continue
            best_vec = rls/ip
            best_error = abs(ip - 1)
            best_lcm = ip.as_integer_ratio()[1]
    return best_vec


#helper function, to add the kernel constraints to the table constructor
def adjust_table_constructor(self, table_constructor, y_vector):
    for param in table_constructor.keys():
        ns, ftype = param
        table = self.mul_project_table(ns, ns, ftype, [])
        
        Zs = [None]*len(table_constructor[param])
        for gg, morig in enumerate(table):
            for ii, base in enumerate(table_constructor[param]):
                mat = base * morig * base.T
                if Zs[ii]==None:
                    Zs[ii] = mat*y_vector[gg]
                else:
                    Zs[ii] += mat*y_vector[gg]
        
        new_bases = []
        for ii, Z in enumerate(Zs):
            if min(Z.eigenvalues())<0:
                print("Construction based Z matrix for {} is not semidef: {}".format(ftype, min(Z.eigenvalues())))
            Zkern = Z.kernel().basis_matrix()
            if Zkern.nrows()>0:
                new_bases.append(Zkern * table_constructor[param][ii])
        table_constructor[param] = new_bases

    return table_constructor

#helper function, moves a table to an sdp input data
def tables_to_sdp_data(self, table_constructor, prev_data=None):
    if prev_data==None:
        mat_inds = []
        mat_vals = []
        block_sizes = []
    else:
        mat_inds, mat_vals, block_sizes = prev_data
    block_index = len(block_sizes) + 1
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        block_sizes += [base.nrows() for base in table_constructor[params]]
        
        #only loop through the table once
        for gg, morig in enumerate(table):
            #for each base change create the entries
            for plus_index, base in enumerate(table_constructor[params]):
                mm = base * morig * base.T
                dd = mm._dict()
                if len(dd)>0:
                    inds, values = zip(*mm._dict().items())
                    iinds, jinds = zip(*inds)
                    for cc in range(len(iinds)):
                        if iinds[cc]>=jinds[cc]:
                            mat_inds.extend([gg+1, block_index + plus_index, iinds[cc]+1, jinds[cc]+1])
                            mat_vals.append(values[cc])
        block_index += len(table_constructor[params])
    return mat_inds, mat_vals, block_sizes

#helper function, moves the linear constraints to the sdp input data
def constraints_to_sdp_data(self, flag_num, constraints_vals, constraints_flags_vec, prev_data=None):
    if prev_data==None:
        mat_inds = []
        mat_vals = []
        block_sizes = []
    else:
        mat_inds, mat_vals, block_sizes = prev_data
    block_index = len(block_sizes) + 1

    constr_num = len(constraints_vals)
    
    for ii in range(constr_num):
        mat_inds.extend([0, block_index+1, 1+ii, 1+ii])
        mat_vals.append(constraints_vals[ii])
    
    for gg in range(flag_num):
        mat_inds.extend([gg+1, block_index, gg+1, gg+1])
        mat_vals.append(1)
        for ii in range(constr_num):
            mat_inds.extend([gg+1, block_index+1, ii+1, ii+1])
            mat_vals.append(constraints_flags_vec[ii][gg])
    block_sizes += [-flag_num, -constr_num]
    
    return mat_inds, mat_vals, block_sizes

#flatten and unflatten symmetric matrices
def flatten_matrix(mat, doubled=False):
    res = []
    factor = 2 if doubled else 1
    for ii in range(len(mat)):
        res.append(mat[ii][ii])
        res += [factor*mat[ii][jj] for jj in range(ii+1, len(mat))]
    return res

def unflatten_matrix(ls, dim, doubled=False):
    mat = [[0]*dim for ii in range(dim)]
    factor = 2 if doubled else 1
    index = 0
    for ii in range(dim):
        # Fill the diagonal element
        mat[ii][ii] = ls[index]
        index += 1
        # Fill the off-diagonal elements
        for jj in range(ii + 1, dim):
            mat[ii][jj] = ls[index] / factor
            mat[jj][ii] = ls[index] / factor
            index += 1
    return matrix(mat), ls[index:]

#round the sdp output based on slacks
def round_sdp_solution(self, sdp_result, table_constructor, block_sizes, target_vector_exact, phi_vector_exact, positives_matrix_exact):
    
    flags_num = -block_sizes[-2] # same as |F_n|
    
    c_vector_approx = vector(sdp_result['X'][-2]) # dim: |F_n|, c vector, primal slack for flags
    c_vector_rounded = vector(sdem_list(c_vector_approx)) # as above but rounded

    # The F (FF) flag indecies where the c vector is zero/nonzero
    c_zero_inds = [FF for FF, xx in enumerate(c_vector_approx) if (abs(xx)<1e-6 or phi_vector_exact[FF]!=0)]
    c_nonzero_inds = [FF for FF in range(flags_num) if FF not in c_zero_inds]
    
    
    
    positives_num = -block_sizes[-1] # same as m, number of positive constraints
    phi_pos_vector_exact = positives_matrix_exact*phi_vector_exact # dim: m, witness that phi is positive

    e_vector_approx = vector(sdp_result['X'][-1]) # dim: m, the e vector, primal slack for positivitives
    e_vector_rounded = vector(sdem_list(e_vector_approx)) # as above but rounded
    
    # The f (ff) positivity constraints where the e vector is zero/nonzero
    e_zero_inds = [ff for ff, xx in enumerate(e_vector_approx) if (abs(xx)<1e-6 or phi_pos_vector_exact[ff]!=0)]
    e_nonzero_inds = [ff for ff in range(flags_num) if ff not in e_zero_inds]
    
    
    bound_exact = target_vector_exact*phi_vector_exact # the u value, the bound we want to prove
    # the constraints for the flags that are exact
    corrected_target_relevant_exact = vector([phi_vector_exact[FF] - bound_exact for FF in c_zero_inds])
    # the d^f_F matrix, but only the relevant parts for the rounding
    # so F where c_F = 0 and f where e_f != 0
    positives_matrix_relevant_exact = matrix(QQ, [[positives_matrix_exact[ff][FF] for FF in c_zero_inds] for ff in e_nonzero_inds])
    # the e vector, but only the nonzero entries
    e_nonzero_list_rounded = [e_vector_rounded[ff] for ff in e_nonzer_inds]
    
    # 
    # Flatten the matrices relevant for the rounding
    # 
    # M table transforms to a matrix, (with nondiagonal entries doubled)
    # only the FF index matrices corresponding with tight constraints are used
    # 
    # X transforms to a vector
    # only the semidefinite blocks are used
    # 
    
    # The relevant entries of M flattened to a matrix this will be indexed by 
    # c_zero_inds and the triples from the types
    M_flat_relevant_matrix_exact = matrix(QQ, len(c_zero_inds), 0, 0, sparse=True)
    X_flat_vector_rounded = [] # The rounded X values flattened to a list
    block_index = 0
    block_info = []
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        
        for plus_index, base in enumerate(table_constructor[params]):
            block_info.append([ftype, base])
            X_approx = sdp_result['X'][block_index + plus_index]
            X_flat_vector_rounded += sdem_list(flatten_matrix(X_approx))
            
            M_extra = []
            
            for FF in c_zero_inds:
                M_FF = table[FF]
                M_extra.append(flatten_matrix((base * M_FF * base.T).rows(), doubled=True))
            
            M_flat_relevant_matrix_exact = M_flat_relevant_matrix_exact.augment(matrix(M_extra))
        block_index += len(table_constructor[params])
    
    # 
    # Append the relevant M matrix and the X with the additional values from
    # the positivity constraints. 
    #
    # Then correct the x vector values
    # 
    
    M_matrix_final = M_flat_relevant_matrix_exact.augment(positives_matrix_relevant_exact)
    x_vector_final = vector(X_flat_vector_rounded+e_nonzero_list_rounded)

    # correct the values of the x vector, based on the minimal L_2 norm
    x_vector_corr = x_vector_final - M_matrix_final.T * \
    (M_matrix_final * M_matrix_final.T).pseudoinverse() * \
    (M_matrix_final*x_vector_final - corrected_target_relevant_exact)
    
    #
    # Recover the X matrices and e vector from the corrected x
    #
    
    linear_slacks_corrected = Xflat_corr[:len(linear_slacks_inds)]
    linear_vals_corrected = vector(QQ, linear_vals_num, dict(zip(linear_slacks_inds, Xflat_corr)))
    if min(linear_slacks_corrected)<0:
        print("Linear coefficient is negative: {}".format(min(linear_slacks_corrected)))
    X_final = []
    
    Xflat_corr = Xflat_corr[len(linear_slacks_inds):]
    for ii, dd in enumerate(block_sizes):
        if dd<0:
            break
        Xdd, Xflat_corr = unflatten_matrix(Xflat_corr, dd)
        X_final.append(matrix(Xdd))
        if min(Xdd.eigenvalues())<0:
            print("Rounded X matrix for {} is not semidefinite: {}".format(ftype_list[ii], min(Xdd.eigenvalues())))
    X_final.append(linear_vals_corrected)
    
    
    #
    # Verify the bound and semidefiniteness
    #

    block_index = 0
    slacks = vector(avals) - linear_coeffs_all.T*linear_vals_corrected
    
    for params in table_constructor.keys():
        ns, ftype = params
        table = self.mul_project_table(ns, ns, ftype, [])
        
        for plus_index, base in enumerate(table_constructor[params]):
            Xflat = vector(flatten_matrix(X_final[block_index + plus_index].rows()))
            
            for gg, morig in enumerate(table):
                mm = base * morig * base.T
                matflat = flatten_matrix((base * morig * base.T).rows(), doubled=True)
                slacks[gg] -= vector(matflat)*Xflat
        
        block_index += len(table_constructor[params])
    
    return min(slacks), X_final, Xorig

#the combined code
def optimize_problem(self, target_element, target_size, maximize=True, positives=None, construction=None, certificate=False):
    from csdpy import solve_sdp
    from tqdm import tqdm
    
    #
    # initial setup
    #

    base_flags = self.generate_flags(target_size)
    print("base flags generated")
    mult = -1 if maximize else 1
    avals = (target_element.project()*(mult)<<(target_size - target_element.size())).values()
    print("avals constructed")
    
    #
    # create the table data
    #
    
    ftype_flags = [flag for kk in range(2-target_size%2, target_size-1, 2) 
              for flag in self.generate_flags(kk)]
    ftypes = [flag.subflag([], ftype_points=list(range(flag.size()))) \
              for flag in ftype_flags]
    print("ftypes constructed")
    flags = [self.generate_flags((target_size + ftype.size())//2, ftype) for ftype in ftypes]
    flag_sizes = [len(xx) for xx in flags]
    print("typed flags constructed, their length is {}".format(flag_sizes))
    
    sym_asym_mats = [sym_asym_bases(self, (target_size + ftype.size())//2, ftype) for ftype in ftypes]
    print("sym and asym matrices constructed")
    
    table_constructor = {}
    for ii, ftype in (pbar := tqdm(enumerate(ftypes))):
        ns = (target_size + ftype.size())//2
        #pre-calculate the table here
        table = self.mul_project_table(ns, ns, ftype, [])
        sym_base, asym_base = sym_asym_mats[ii]
        bases = []
        if sym_base.nrows()!=0:
            bases.append(sym_base)
        if asym_base.nrows()!=0:
            bases.append(asym_base)
        table_constructor[(ns, ftype)] = bases
        pbar.set_description("done with mult table for {}".format(ftype))
    print("tables constructed")
    
    sdp_data = tables_to_sdp_data(self, table_constructor)
    print("tables added to sdp data")
    
    #
    # add constraints data
    #
    
    if positives == None:
        constraints_flags = []
        constraints_vals = []
    else:
        constraints_flags = []
        for ii in range(len(positives)):
            fv = positives[ii]
            if isinstance(fv, Flag):
                continue
            d = target_size - fv.size()
            k = fv.ftype().size()
            terms = fv.afae().parent().generate_flags(k+d)
            constraints_flags += [fv.mul_project(xx) for xx in terms]
            print("done with constraint for \n{}\n".format(fv))
        constraints_vals = [0]*len(constraints_flags)
    one_vector = target_element.ftype().project()<<(target_size - target_element.ftype().size())
    constraints_flags.extend([one_vector, one_vector*(-1)])
    constraints_vals.extend([1, -1])
    constraints_flags_vec = [(xx<<(target_size-xx.size())).values() for xx in constraints_flags]
    print("constraints constructed")
    
    sdp_data = constraints_to_sdp_data(self, len(base_flags), constraints_vals, constraints_flags_vec, sdp_data)
    print("constraints added to sdp data")
    
    #
    # if no y value provided, run the optimizer first, only to get the y values
    #
    if construction==None or len(construction)!=len(base_flags) or min(construction.values())<0 or sum(construction.values())!=1:
        print("running sdp for initial y vector")
        mat_inds, mat_vals, block_sizes = sdp_data
        initial_sol = solve_sdp(block_sizes, list(avals), mat_inds, mat_vals)
        y_original = initial_sol['y']
        y_rounded = cfr_adaptive(initial_sol['y'], one_vector.values())
        print("rounded y vector is: \n{}".format(print_sparse(y_rounded)))
    else:
        y_rounded = construction.values()
    
    #
    # adjust the table to consider the kernel from y_rounded
    #
    
    table_constructor = adjust_table_constructor(self, table_constructor, y_rounded)
    sdp_data = tables_to_sdp_data(self, table_constructor)
    sdp_data = constraints_to_sdp_data(self, len(base_flags), constraints_vals, constraints_flags_vec, sdp_data)
    mat_inds, mat_vals, block_sizes = sdp_data
    
    print("running SDP after kernel correction with block sizes {}".format(block_sizes))
    final_sdp = solve_sdp(block_sizes, list(avals), mat_inds, mat_vals)
    
    res = round_sdp_solution(self, final_sdp, table_constructor, avals, y_rounded, block_sizes, constraints_flags_vec)
    if maximize:
        res = (-res[0], res[1], res[2])
    if certificate:
        return res
    else:
        return res[0]

In [9]:
m = matrix(QQ, 3, 0, 0, sparse=True)
m.augment(matrix(QQ, 3, 2))

[0 0]
[0 0]
[0 0]

In [6]:
G = GraphTheory
G.exclude(G(5))
optimize_problem(G, G(2), 5)

base flags generated
avals constructed
ftypes constructed
typed flags constructed, their length is [6, 8, 8, 8, 8]
sym and asym matrices constructed


done with mult table for Ftype on 3 points with edges=[[0, 1], [0, 2], [1, 2]]: : 5it [00:00, 452.17it/s]

tables constructed
tables added to sdp data
constraints constructed
constraints added to sdp data
running sdp for initial y vector
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 1.00e+00 Pobj: -2.2612306e+01 Ad: 7.05e-01 Dobj: -3.9541750e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -2.2826131e+01 Ad: 9.52e-01 Dobj: -4.3149319e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.7548649e+01 Ad: 9.21e-01 Dobj: -4.5976664e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -5.3782398e+00 Ad: 6.78e-01 Dobj: -4.6615539e-01 
Iter:  5 Ap: 6.03e-01 Pobj: -4.0591390e+00 Ad: 9.28e-01 Dobj: -4.7029138e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.2337085e+00 Ad: 7.19e-01 Dobj: -4.8536048e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -9.3548538e-01 Ad: 6.98e-01 Dobj: -5.5559389e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -8.3047196e-01 Ad: 7.37e-01 Dobj: -6.3863527e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -7.7922959e-01 Ad: 7.61e-01 Dobj: -6.9060569e-01 
Iter: 10 Ap: 1.00e+00 Pobj: -7.5819260e-01 Ad: 8.48e-01 Dobj: -7.2647700e-01 


rounded y vector is: 
0: 15/64
4: 45/128
7: 15/64
19: 15/128
26: 15/256
32: 1/256
Running SDP after kernel correction with block sizes [3, 1, 3, 3, 5, 2, 6, 2, 3, 4, -33, -2]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 9.60e-01 Pobj: -1.9063954e+01 Ad: 7.17e-01 Dobj:  9.9599339e-01 
Iter:  2 Ap: 1.00e+00 Pobj: -2.1206090e+01 Ad: 9.13e-01 Dobj:  1.0318649e-01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.9858279e+01 Ad: 9.34e-01 Dobj: -4.1109804e-01 
Iter:  4 Ap: 1.00e+00 Pobj: -7.8756166e+00 Ad: 7.57e-01 Dobj: -4.3347789e-01 
Iter:  5 Ap: 1.00e+00 Pobj: -4.1206975e+00 Ad: 8.63e-01 Dobj: -4.4400536e-01 
Iter:  6 Ap: 1.00e+00 Pobj: -1.3854918e+00 Ad: 7.97e-01 Dobj: -4.6135858e-01 
Iter:  7 Ap: 1.00e+00 Pobj: -8.6507073e-01 Ad: 7.54e-01 Dobj: -5.1120012e-01 
Iter:  8 Ap: 1.00e+00 Pobj: -8.2362572e-01 Ad: 7.23e-01 Dobj: -6.1401112e-01 
Iter:  9 Ap: 1.00e+00 Pobj: -7.8762708e-01 Ad: 7.27e-01 Dobj: -6.7838001e-01 
Iter: 10 Ap: 1.00e+00 Pobj: -7.562

NameError: name 'no_slack_inds' is not defined

In [ ]:
import itertools

# a general identifier code that works on any number of colors and edge arity
def _identifyCT(k, order_partition, n, ftype_points, **kwargs):
    is_graph = (k==2)
    color_number = sum(len(xx) for xx in order_partition)
    edges = kwargs["edges"]
    Cs = [[cx[0] for cx in kwargs["C{}".format(ii)]] for ii in range(color_number)]
    ftype_union = [jj for ff in ftype_points for jj in ff]
    g_parts = list(ftype_points) + \
              [[ii for ii in range(n) if ii not in ftype_union]]
    ppadd = 0 if is_graph else len(edges)
    g_verts = list(range(n+ppadd+color_number))
    g_parts.append(list(range(n, n+ppadd)))

    g_parts += [[n+ppadd+ii for ii in partition_j] for partition_j in order_partition]
    
    if is_graph:
        g_edges = list(edges)
        for ii in range(color_number):
            g_edges += [(xx, n+ii) for xx in Cs[ii]]
    else:
        g_edges = [(i+n,x) for i,b in enumerate(edges) for x in b]
        for ii in range(color_number):
            g_edges += [(xx, n+len(edges)+ii) for xx in Cs[ii]]
    g = Graph([g_verts, g_edges], format='vertices_and_edges')
    blocks = tuple(g.canonical_label(partition=g_parts).edges(labels=None, sort=True))
    return (n, tuple([len(xx) for xx in ftype_points]), blocks)

# a general generator code that also works on any number of colors and edge arity
def _generateCT(k, order_partition, n):
    color_number = sum(len(xx) for xx in order_partition)
    if k==2:
        BT = GraphTheory
    if k==3:
        BT = ThreeGraphTheory
    for xx in BT.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        
        for yy in itertools.product(range(color_number), repeat=int(n)):
            yy = list(yy)
            Cs = {"C{}".format(cc):[[ii] for ii, oo in enumerate(yy) if oo==cc] for cc in range(color_number)}
            iden = _identifyCT(k==2, order_partition, n, [], edges=edges, **Cs)
            if iden not in unique:
                unique.append(iden)
                Cs["edges"] = edges
                yield Cs

#to make a resulting theory create the functions directly with the correct number of parameters:

#for example, to make graphs with 2 unordered colors
#we say k=2 to talk about regular graphs
#and order_partition=[[0, 1]] to say that 0 and 1 can be interchanged (if you want ordered colors then the partition is [[0], [1]])
#edges, C0, C1 are the relations
def test_identify(n, ftype_points, edges, C0, C1):
    return _identifyCT(2, [[0, 1]], n, ftype_points, edges=edges, C0=C0, C1=C1)

#same for the generator. k=2 and order_partition=[[0, 1]].
def test_generate(n):
    return _generateCT(2, [[0, 1]], n)

TTT = CombinatorialTheory("2UnOrdColGraph", test_generate, test_identify, edges=2, C0=1, C1=1)

tri_000 = TTT(3, edges=[[0, 1], [0, 2], [1, 2]], C0=[[0], [1], [2]])
tri_001 = TTT(3, edges=[[0, 1], [0, 2], [1, 2]], C0=[[0], [1]], C1=[[2]])
TTT.exclude([tri_000, tri_001])

Bp = TTT(2, edges=[[0, 1]], C0=[[0], [1]], ftype=[0])
Gp = TTT(2, edges=[[0, 1]], C0=[[0]], C1=[[1]], ftype=[0])

G = TTT(2, edges=[[0, 1]], C0=[[0]], C1=[[1]])
B = TTT(2, edges=[[0, 1]], C0=[[0], [1]])
M = TTT(2, C0=[[0]], C1=[[1]])

optim = blowup_construction(TTT, 5, 2, edges=[[0, 1]], C0=[[0]], C1=[[1]])

res = optimize_problem(TTT, B - M, 5, positives=[Gp-Bp/2, G-1/3], construction=optim, certificate=True)